# 4. Procesado de datos.

## 4.2 Creación de conjunto de entrenamiento y pruebas. 

In [3]:
# Importando librerias.
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Importando datasets from datafiles.
mainpath = "/proyectos/learning/MachineLearning/datasets/aprendizaje-autom-profundo/"
outputs = "/proyectos/learning/MachineLearning/datasets/aprendizaje-autom-profundo/outputs/"
filename = "precios_casas.csv"
fullpath = os.path.join(mainpath, filename)

In [5]:
# Creando conjunto con biblioteca Scikit-learn funcion train_test_split().

"""
X = atributos descriptivos.
y = valores de la etiqueta.
entX = conjunto entrenamiento atributos descriptivos (3.680 x 16)
pruX = conjunto entrenamiento atributo etiqueta (3.680)
enty = conjunto de prueba atributos descriptivos (920 x 16)
pruy = conjunto de prueba atributo etiqueta (920)
test_size=0.2 = 20%
random_state=100 = Se usa como semilla para inicializar un objeto generador de numeros aleatorios.
"""

import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv(fullpath)
X = df.iloc[:,1:].values
y = df.iloc[:,0].values
print(X)
entX, pruX, enty, pruy = train_test_split(X, y, test_size=0.2, random_state=100)

[[3.0 1.5 1340 ... 'Shoreline' 'WA 98133' 'USA']
 [5.0 2.5 3650 ... 'Seattle' 'WA 98119' 'USA']
 [3.0 2.0 1930 ... 'Kent' 'WA 98042' 'USA']
 ...
 [3.0 2.5 3010 ... 'Renton' 'WA 98059' 'USA']
 [4.0 2.0 2090 ... 'Seattle' 'WA 98178' 'USA']
 [3.0 2.5 1490 ... 'Covington' 'WA 98042' 'USA']]


In [4]:
# Rotorno de valores ausentes por columna.
df.isnull().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64

In [6]:
df = pd.DataFrame([
    ['1',1,30],
    ['2',1,32],
    ['3',0]],
)
df.columns = ['codigo', 'credito', 'edad']
df

,codigo,credito,edad
0,1,1,30.0
1,2,1,32.0
2,3,0,NaN


In [7]:
df.isnull().sum()

codigo     0
credito    0
edad       1
dtype: int64

In [8]:
# Remplazando caracteres especiales por NaN.
df.replace('?',np.nan, inplace=True)

In [10]:
# Imputando el codigo, posterior al remplazo de caracteres.
# se remplaza el valor faltante por 31. Equivalente a la medida aritmetica de la 3ra columna (30+32)/2 = 31
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(df.values)
imp_datos = imp.transform(df.values)
imp_datos

array([[ 1.,  1., 30.],
       [ 2.,  1., 32.],
       [ 3.,  0., 31.]])

## 4.4 Manejo de datos categoricos.
- Convertir a datos numericos los datos categoricos

In [69]:
df = pd.DataFrame([
    ['M',30,'Amarillo', 'Clase 1'],
    ['P',28,'Azul','Clase 2'],
    ['J',21,'Rojo','Clase 1'],
    ['J',21,'Verde','Clase 1']],
)
df.columns = ['nombre','edad','color','etiqueta']
dfAux = df
df2 = df
df3 = df

In [70]:
# Convirtiendo los valores de la columna etiqueta a su representacion numerica.
from sklearn.preprocessing import LabelEncoder
le_clase = LabelEncoder()
y = le_clase.fit_transform(df.etiqueta)
print(y)

[0 1 0 0]


In [71]:
# Implementando el metodo inverso.
clase_inv = le_clase.inverse_transform(y)
print(clase_inv)

['Clase 1' 'Clase 2' 'Clase 1' 'Clase 1']


In [74]:
# Usando la tecnica ONE HOT ENCODING.
from sklearn.preprocessing import OneHotEncoder
le_color = LabelEncoder()
ohe_color = OneHotEncoder(categories='auto')
df['color_cod'] = le_color.fit_transform(df.color)
print(df)


  nombre  edad     color etiqueta  color_cod
0      M    30  Amarillo  Clase 1          0
1      P    28      Azul  Clase 2          1
2      J    21      Rojo  Clase 1          2
3      J    21     Verde  Clase 1          3


In [75]:
# Concatenando columnas color_cod con el dataframe.
datos_ohe = ohe_color.fit_transform(df.color_cod.values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(datos_ohe, columns=['Color_'+str(int(i)) for i in range(len(df.color))])
df2 = pd.concat([df, dfOneHot], axis=1)
df2

,nombre,edad,color,etiqueta,color_cod,Color_0,Color_1,Color_2,Color_3
0,M,30,Amarillo,Clase 1,0,1.0,0.0,0.0,0.0
1,P,28,Azul,Clase 2,1,0.0,1.0,0.0,0.0
2,J,21,Rojo,Clase 1,2,0.0,0.0,1.0,0.0
3,J,21,Verde,Clase 1,3,0.0,0.0,0.0,1.0


In [76]:
# Eliminando columnas color y color_cod.
df2 = df2.drop(['color', 'color_cod'], axis=1)
df2

,nombre,edad,etiqueta,Color_0,Color_1,Color_2,Color_3
0,M,30,Clase 1,1.0,0.0,0.0,0.0
1,P,28,Clase 2,0.0,1.0,0.0,0.0
2,J,21,Clase 1,0.0,0.0,1.0,0.0
3,J,21,Clase 1,0.0,0.0,0.0,1.0


In [77]:
# Usando funcion get_dummies para convertir variables categoricas a variables indicadoras.
pd.get_dummies(data=dfAux, columns=['color'],drop_first=True)
df3 = df3.drop(['color_cod'], axis=1)
df3

,nombre,edad,color,etiqueta
0,M,30,Amarillo,Clase 1
1,P,28,Azul,Clase 2
2,J,21,Rojo,Clase 1
3,J,21,Verde,Clase 1
